# IMPORTING LIBRARIES & STARTING EARTH ENGINE

## SETUP AND DATA COLLECTION CHOOSING THE GEOGRAPHIC SPACE ( Cork )
According to 
The irish times: https://www.irishtimes.com/ireland/2023/10/19/cork-flooding-floods-in-co-cork-absolutely-devastating-as-safety-warning-issued-to-motorists/
Picked the area that got more affected with the floods of 2023


In this phase, we are setting up Google Earth Engine to fetch and process satellite images for a specific area and time range. 

Defining the Area of Interest (AOI): Specifying the geographic region.
Fetching Satellite Images: Retrieving Landsat and Sentinel-2 images for the defined AOI and time range, with filters for cloud cover.
Visualizing the Data: Adding the median composite images to an interactive map for visualization.
Exporting the Data: Exporting the processed images to Google Drive for further analysis.
This phase is crucial for gathering the necessary data to analyze changes in water-covered areas, which is essential for flood detection.

Setting up the environment


In [43]:
import ee
import geemap

# Initialize GEE
ee.Authenticate()
ee.Initialize(project='atu-fyp')

Setting Area choosen

In [44]:
# Define AOI for Cork
aoi = ee.Geometry.Polygon([
    [-8.570156845611686, 52.00904254772663],
    [-8.570156845611686, 51.7292195887807],
    [-7.959042343658562, 51.7292195887807],
    [-7.959042343658562, 52.00904254772663]
]).buffer(5000)  # Buffer to expand the area

# Initialize the map
Map = geemap.Map(center=[51.8691, -8.2646], zoom=10)
Map.addLayer(aoi, {'color': 'red'}, 'Area of Interest')
Map

Map(center=[51.8691, -8.2646], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

Loading Sentinel-1 Data
Load Sentinel-1 GRD data for pre-flood and post-flood periods.

In [45]:
# Defines pre-flood and post-flood dates
pre_flood_start = '2023-09-01'
pre_flood_end = '2023-09-30'
post_flood_start = '2023-10-01'
post_flood_end = '2023-10-31'

# Loads Sentinel-1 GRD data for pre-flood period
pre_flood_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(pre_flood_start, pre_flood_end) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select('VV')

# Loads Sentinel-1 GRD data for post-flood period
post_flood_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select('VV')

# Gets the median image for each period
pre_flood_image = pre_flood_collection.median()
post_flood_image = post_flood_collection.median()

# Adds pre-flood and post-flood images to the map
Map.addLayer(pre_flood_image.clip(aoi), {'min': -20, 'max': 0}, 'Pre-Flood VV')
Map.addLayer(post_flood_image.clip(aoi), {'min': -20, 'max': 0}, 'Post-Flood VV')
Map.centerObject(aoi)
Map

Map(bottom=87045.0, center=[51.868860004634016, -8.265151977539064], controls=(WidgetControl(options=['positio…

 Calculate the Difference Between Pre-Flood and Post-Flood Images
 Subtracting the pre-flood image from the post-flood image to detect changes.

In [46]:
# Calculates the difference between post-flood and pre-flood images
difference_image = post_flood_image.subtract(pre_flood_image)

# Adds the difference image to the map
Map.addLayer(difference_image.clip(aoi), {'min': -5, 'max': 5, 'palette': ['red', 'white', 'blue']}, 'Difference Image')
Map.centerObject(aoi)
Map

Map(bottom=87044.0, center=[51.86970795368951, -8.265151977539064], controls=(WidgetControl(options=['position…

Thresholding to Identify Flooded Areas
Applying a threshold to the difference image to identify flooded areas.

In [47]:
# Define a threshold for flood detection
threshold = 2  # Adjust this value based on your analysis

# Create a binary flood mask
flood_mask = difference_image.gt(threshold)

# Add the flood mask to the map
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'Flood Mask')
Map.centerObject(aoi)
Map

Map(bottom=87044.0, center=[51.86970795368951, -8.265151977539064], controls=(WidgetControl(options=['position…